# Guided Mutations

So far, we have essentially created inputs _randomly_, in the hope that these may eventually trigger a bug.  If we can feed in knowledge about the program behavior, we can be much more efficient in creating useful inputs.  The key to success lies in the idea of _guiding_ these mutations – that is, _keeping those that are especially valuable._  "Valuable" might mean that they would be syntactically valid, but also that they help covering more implemented behavior.

**Prerequisites**

* This chapter extends our previous implementation of mutation-based fuzzing from the ["Mutation-Based Fuzzing"](Mutation_Based_Fuzzing.ipynb) chapter.
* You should understand how to measure code coverage; for instance from the ["Coverage"](Coverage.ipynb) chapter.

## Multiple Mutations

So far, we have only applied one single mutation on a sample string.  However, we can also apply _multiple_ mutations, further changing it.  What happens, for instance, if we apply, say, 20 mutations on our sample string?

In [31]:
import gstbook

In [32]:
%%capture
from Mutation_Fuzzing import mutate, is_valid_expr, valid_inputs

In [33]:
seed_input = "1 + 2 * 3 / 4"
MUTATIONS = 20
inp = seed_input

In [34]:
for i in range(MUTATIONS):
    inp = mutate(inp)
    print(repr(inp))

'1 + 2 * 3 / .4'
'! + 2 * 3 / .4'
'! + 2 * 3 / ,4'
' ! + 2 * 3 / ,4'
' ! + 2 %* 3 / ,4'
' ! + 2 %* 3 /2 ,4'
' # + 2 %* 3 /2 ,4'
' ! + 2 %* 3 /2 ,4'
' ! + 2% %* 3 /2 ,4'
' ! + 2% %* 3 ?2 ,4'
' ! + 2% %* 3 ?2,4'
' ! + 2%< %* 3 ?2,4'
' ! + 2%=< %* 3 ?2,4'
'! + 2%=< %* 3 ?2,4'
'! + 2%=< % 3 ?2,4'
'! + 2%=< %3 ?2,4'
'!+ 2%=< %3 ?2,4'
'!:+ 2%=< %3 ?2,4'
'!:+ 2%=< % ?2,4'
'!:+2%=< % ?2,4'


As you see, the original seed input is hardly recognizable anymore.  Mutating the input again and again has the advantage of getting a higher variety in the input, but on the other hand further increases the risk of having an invalid input.

The key to success lies in the idea of _guiding_ these mutations – that is, _keeping those that are especially valuable._  For instance, we may want to keep those samples that happen to be _valid_, and keep on mutating these.  To this end, we proceed as follows:

1. We keep a population of seen valid inputs `population`, initialized with the seed input.
2. We pick a random (valid) candidate from this set, which we now mutate up to `MAX_MUTATIONS` times.
3. If the candidate is valid and new, we add it to `population`.
4. In the next iterations, the previous candidates can be chosen again as seeds.

Here's the full code:

In [35]:
import random

In [36]:
def create_candidate(population):
    MIN_MUTATIONS = 2
    MAX_MUTATIONS = 10

    candidate = random.choice(population)
    trials = random.randint(MIN_MUTATIONS, MAX_MUTATIONS)
    for i in range(trials):
        candidate = mutate(candidate)
    return candidate

In [37]:
def create_population(seed, size):
    population = seed
    while len(population) < size:
        candidate = create_candidate(population)
        if is_valid_expr(candidate) and candidate not in valid_inputs:
            population.append(candidate)
    return population

In [38]:
seed_input = "1 + 2 * 3 / 4"
POPULATION_SIZE = 20
create_population(seed=[seed_input], size=POPULATION_SIZE)

['1 + 2 * 3 / 4',
 '1 + 3 * 3/ 5',
 '1 + 3 * 3/4',
 '-1- + 3 * 8/ 5',
 'm1- + 3 * 8+ 5',
 '-1- + 3 *0/ 5',
 'm1--+ 3 * 8+ 5',
 '0+ 3 * 3/ 5',
 '1>+ 3= +23/ 5',
 'mq-3+ 3 * 8+ 5',
 ' 3+ 2 * 3 / 4',
 'm1- + 3  < 5',
 'q & 2   / 4',
 'o%1- + 3  < 5',
 'q & 2   / 4',
 '1.- +3 * 81/ 5',
 ' * 3 / 5',
 '3>+3=+23/ 5',
 '-1-  5+ 3 * 8/ 5',
 '-1- +3 *0/ 85']

As you can see, we now get an even larger variety of inputs.  If you are using the interactive notebook version of this chapter, you can toy with different settings (such as `MAX_MUTATION`) to see how they influence the outputs.  (Also see the [Exercises](#Exercises) below.)

## Maximizing Diversity

A good set of tests not only consists of valid inputs, but the inputs should also be _as diverse as possible_: If you already have tested `1 + 1`, then testing for `1 + 2` or `1 + 1 + 1` is not going to cover much additional functionality, compared to, say, `15.0 / 7`.  To easily ensure a greater diversity, we can check for two features: _length_ and _input elements_.


### Diversity through Length

A simple way to increase diversity in our setting is ro favor _longer inputs_.  The argument is simple: The longer a randomly generated input is, the higher its chances to cover several program features.  Let us define a function `evolve_population()` that 

1. Sorts the population in ascending order according to its _fitness_ (the length)
2. Replaces the first element in the population by a new, fitter (longer) one

This is what this looks like:

In [50]:
def evolve_population(population, fitness=len, steps=100):
    evolutions = 0
    while evolutions < steps:
        population.sort(key=fitness)

        candidate = create_candidate(population)
        if is_valid_expr(candidate) and fitness(candidate) > fitness(population[0]):
            population[0] = candidate
            evolutions += 1

    return population

Let us try out `evolve_population()` with a sample population in which one element `25` is the shortest and thus would be replaced by a new, longer element:

In [55]:
evolve_population(population=['1 + 1', '1 + 1 + 1', '25'], steps=1)

['32 +1', '1 + 1', '1 + 1 + 1']

We can now use this to evolve a population over several iterations.  We can see that the strings get longer and longer:

In [51]:
population = create_population(seed=[seed_input], size=POPULATION_SIZE)
evolve_population(population)

["(-44)+' z*593(+q8\x0f?.84',",
 '(34)+\' 0z2533+q8!"m.4\'',
 "(44)+' z7*593+q8\x0f8.4',",
 "(34 )+'2z7*?5>9;3=\x0f<.4'",
 "(24 )+'2z7*5>9;38=\x0f<.4'",
 "(45)+' z7*5;\x13+!q8\x0f8.4',",
 "(64)+'2z7*1>9338=\x0f<3.4'",
 "(34)+' 0zj2530+q8!2o.4'",
 ' 34.++\' 05z243;q9"!"m.4\'',
 '34+\'505-z25*33kq490!"m4\'',
 "(34)+' 0zj2%530+q8!\x12o.4'",
 "(-44)+' z7*593(+q8\x0f.84',",
 "(304)+' 0zj28730+8!2o.t'",
 '(34)+\' 505z2533+q9!"m.4\'',
 "(304)+' 0zj2530+q8!2o.t'",
 "(34 )+':2z7*?7>1;3=4\x0e<4'",
 "(34 )*'/:3z3*?\x1e>193=4\x0e\x1c5'",
 '(34)+\' 0zj2530+q8!"7>o.4\'',
 "(34 )+'2z7*/5>79.;3=\x0f<.,'",
 "(4.594)+' ~72=)o3+q82*o.\x14'"]

### Diversity in Input Elements

Not only do we want long inputs, but we also want diversity in input elements.  For instance, we'd like our inputs to cover as many arithmetic operations as possible: The input `1 + 2 * 3` is shorter than the input `1234567890`, yet covers more operations.  We can achieve this by defining a special fitness function that favors inputs with a large diversity in input characters.

In [42]:
def diversity_fitness(s):
    return len(set(s))

Here, `len(set(s))` returns the size of the set of characters in `s` – that is, the number of different characters:

In [43]:
diversity_fitness("1 + 1")

3

We can see that this gives us even more diverse inputs:

In [52]:
evolve_population(population, fitness=diversity_fitness)

['.6-(363425 )+\':*x/*768!"?*7>71;2=0\x0e%,$)\x1cf6%\'',
 "(47630 )+'<9&2z\x017*5/%>97\x19.*\x1a3\x0f$'",
 "1(34 )+'2z7*/5$>%0=.;3=\x0f<-!.(l'",
 '6-(3634 )+\':x*768"?"7>1;q=0\x0e!$)\x1c%\'',
 "2-(34)+':<z 768?>1;s=0\x0e%$*\x1c<%'",
 '6-(334 )+\':*x*768!"?*7~1;s3=0\x0e%$)\x1c6%\'',
 "2-(34 )+':<z *768?7>1;s=0\x0e%$*\x1c%'",
 "2-(334 )+'::x*768?7>1;s=0\x0e!$)\x1c%'",
 "6-(334 )+'::x:768*?7>519q=0\x0e!$)\x1c%'",
 "1(34 )+'2z7*/05$>%0=8.;39\x0f<-%.(l'",
 '6-(334 )+\'::x*768"?"7>1;s=0\x0e!$)\x1c6%\'',
 "1(34 )+'2z7*/05$>%0=.;39\x0e<-!4.(l'",
 "1(34 )+'2z7*/5$>%0.;3=\x0f<-!.)hl'",
 '6-(234 )+\'::x768"?*7>1;s=0\x0e$):\x1c6%\'',
 '6-(334 )+\':x*768"?"7>151;q=0\x0e!$)\x1c%\'',
 '6-(334 )+\'::x*768"?"7>51;q=0\x0e!$)\x1c%\'',
 '2-(34 )+\'::x*768?"7>1;s=0\x0e!$)\x1c%\'',
 '6-(334 )+\'::x*768\x1f">1;s=0+\x0e!$)\x1c6%\',',
 "1(634)+'12zw*/5$6% 0.;73=>\x0f<-!.9hl'",
 '.6-(33425 )+\':*x/*768!"?*7~71;2=0\x0e%,$)\x1c6%\'']

However, there's a catch: As soon as our input contains a comment (a substring starting with `#`), any characters are allowed after the `#`.  The same holds if our input contains a quoted string (`'...'` or `"..."`); then arbitrary characters can be added to the string.  We may thus achieve diversity in comments or strings, but not necessarily in functionality.  

This, of course, is one of the central problems of software testing: How can I cover all functionality of the program under test – including all its bugs?  Fortunately, researchers and practitioners have devised a great number of solutions to address this problem.

## Guiding by Coverage

\todo{add}

## Next Steps


How can we sufficiently cover functionality?  Essentially, we have two options:

1. Try to cover as much _implemented_ functionality as possible.  To this end, we need to access the program implementation, measure which parts would actually be reached with our inputs, and use this _coverage_ to guide our search.  We will explore this in the next chapter, which discusses [coverage](Coverage.ipynb).

2. Try to cover as much _specified_ functionality as possible.  Here, we would need a specification of the input format, distinguishing between individual input elements such as (in our case) numbers, operators, comments, and strings – and attempting to cover as many of these as possible.  We will explore this as it comes to [grammar-based testing](Grammar_Testing.ipynb).

Finally, the concept of a "population" that is systematically "evolved" trhough "mutations" will be explored in depth when discussing [search-based testing](Search_Based_Testing.ipynb).  Enjoy!


## Exercises


### Exercise 1

Apply the above mutation-based fuzzing technique on `bc`, using files, as in our [Introduction to Fuzzing](Basic_Fuzzing.ipynb).

### Exercise 2

To achieve diversity in our set of inputs, one may also try to maximize the _difference_ between the individual inputs in the population.  Rather than sorting the population and replace the least fit individual, we could also compare all individuals against each other and pick the one that is most similar to all the others.  For instance, one could define similarity as the ratio of which characters are common in both strings:

In [45]:
def similar(s1, s2):
    return len(set(s1) & set(s2)) / len(set(s1) | set(s2))

In [53]:
assert similar("A", "A") == 1.0
assert similar("A", "B") == 0

Here, `a & b` and `a | b` denote the intersection and union of two sets, respectively.

The more the set of characters in two strings is the same, the higer the value of `similar()`:

In [46]:
similar("Apple", "Microsoft")

0.0

In [47]:
similar("Python", "Boa constrictor")

0.23076923076923078

In [48]:
similar("Python 2", "Python 3")

0.7777777777777778

When it comes to remove one element from the population, use the above `similar()` implementation to determine the one element that is most similar to all the others.  Report a typical output sample.   How expensive is your approach?

### Exercise 3

Python brings an [impressive library for computing the difference between strings](https://docs.python.org/3/library/difflib.html), which we can use to measure similarity between two strings including the ordering of characters:

In [49]:
from difflib import SequenceMatcher

In [54]:
def similar(s1, s2):
    return SequenceMatcher(None, s1, s2).ratio()

Repeat [Exercise 2](#Exercise-2) with the above.